In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc

In [2]:
# Create various waves
def create_hill(x):
    # Hill as in Wicker and Skamarock (2002)
    return 1. / (1. + np.exp(80.*(np.abs(x-0.5) - 0.15)))
def create_box(x):
    f = np.zeros(x.shape)
    # Box
    f[20:31] = .8
    return f
def create_sawtooth_box(x):
    f = np.zeros(x.shape)
    # Sawtooth Box
    f[20:26] = .8
    f[26:31] = -.8
    return f
def create_triangle(x):
    f = np.zeros(x.shape)
    # Triangle
    f[20:26] = .8/5*np.array([0,1,2,3,4,5])
    f[25:31] = .8-.8/5*np.array([0,1,2,3,4,5])
    return f
def create_sawtooth_triangle(x):
    f = np.zeros(x.shape)
    # Sawtooth Box
    f[15:21] = .8/5*np.array([0,1,2,3,4,5])
    f[20:26] = .8-.8/5*np.array([0,1,2,3,4,5])
    f[25:31] = -.8/5*np.array([0,1,2,3,4,5])
    f[30:36] = -.8+.8/5*np.array([0,1,2,3,4,5])
    return f
def create_sine_wave(x):
    f = np.zeros(x.shape)
    f[15:35] = .8*np.sin(x[15:35]*7*np.pi)
    return f

In [3]:
# First set up the figure, the axis, and the plot element we want to animate
def animate(q,exact_f,x,f,title):
    rc('animation', html='html5')

    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(xlim=(0, 1), ylim=(-1.0, 1.0))
    plt.title(title)
    time_template = 't = %.0f'
    time_text = ax.text(0.05, .9, '')
    ax.axvline(0.5,ls='dotted',color='k')
    ax.text(0.8,.9,'TRER = '+str(np.round(TRER(q,f),decimals=4)))
    line, = ax.plot([], [], lw=2, animated=True)
    line2, = ax.plot([], [], lw=2, ls='dashed', color='r', animated=True)
    #plt.tight_layout()
    plt.close()

    # initialization function: plot the background of each frame
    def init():
        line.set_data([], [])
        line2.set_data([], [])
        time_text.set_text('')
        return (line, line2, time_text)

    # animation function.  This is called sequentially
    def animate(i):
        line.set_data(x, q[i,:])
        line2.set_data(x, exact_f[i,:])
        time_text.set_text(time_template % (i))
        return (line, line2, time_text)

    # call the animator.  blit=True means only re-draw the parts that have changed.
    return animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=251, interval=100, repeat=False,
                                   blit=True)


In [4]:
def forward_upwind(f,cnum):
    # Forward in time, Upwind in space
    q = np.empty((251,51))
    q[0,:] = f.copy()
    
    for i in range(1,251):
        q[i,0]  = q[i-1,0]  - cnum*(q[i-1,0]  - q[i-1,49])      # BCs
        q[i,1:] = q[i-1,1:] - cnum*(q[i-1,1:] - q[i-1,0:-1])    # Upwind Spatial Differencing
    return q

def TRER(q,f):
    return np.sqrt(np.sum((f - q[250,:])**2./50.))

In [5]:
def leapfrog_2nd_order(f,cnum,gamma):
    # Leapfrog in time and space
    qc = np.empty((251+1,51))
    qf = np.empty((251+1,51))
    qc[0,:] = f.copy()
    qf[0,:] = f.copy()

    # First time step
    qc[1,0] = qc[0,0] - cnum*(qc[0,1]-qc[0,49])
    qc[1,1:50] = qc[0,1:50] - cnum*(qc[0,2:51]-qc[0,0:49])
    qc[1,50] = qc[1,0]

    for i in range(2,251+1):
        qc[i,0] = qf[i-2,0] - cnum*2.*(qc[i-1,1] - qc[i-1,48])/2.
        qc[i,1:49] = qf[i-2,1:49] - cnum*2.*(qc[i-1,2:50] - qc[i-1,0:48])/2.
        qc[i,49] = qc[i,0] # BC
        qc[i,50] = qc[i,1] # BC

        # Asselin Filter to control computational mode with Leapfrog in time
        if gamma>0:
            qf[i-1,:] = qc[i-1,:] + gamma*(qc[i,:]-2.0*qc[i-1,:]+qf[i-2,:])
        else:
            qf[i] = qc[i]
    return qf[:251]

In [6]:
def leapfrog_4th_order(f,cnum,gamma):
    qc4 = np.empty((251+1,51))
    qf4 = np.empty((251+1,51))
    qc4[0,:] = f.copy()
    qf4[0,:] = f.copy()

    # First time step, upwind
    qc4[1,0] = qc4[0,0] - cnum*(qc4[0,1]-qc4[0,49])
    qc4[1,1:50] = qc4[0,1:50] - cnum*(qc4[0,2:51]-qc4[0,0:49])
    qc4[1,50] = qc4[1,0]

    for i in range(2,251+1):
        qc4[i,0] = qf4[i-2,0] - cnum*2.*(8.*(qc4[i-1,1]-qc4[i-1,49]) - qc4[i-1,2] + qc4[i-1,48])/12.
        qc4[i,1] = qf4[i-2,1] - cnum*2.*(8.*(qc4[i-1,2]-qc4[i-1,0]) - qc4[i-1,3] + qc4[i-1,49])/12.
        qc4[i,2:49] = qf4[i-2,2:49] - cnum*2.*(8.*(qc4[i-1,3:50]-qc4[i-1,1:48]) - qc4[i-1,4:51] + qc4[i-1,0:47])/12.

        # BCs
        qc4[i,49] = qf4[i-2,49] - cnum*2.*(8.*(qc4[i-1,0]-qc4[i-1,48]) - qc4[i-1,1] + qc4[i-1,47])/12.
        qc4[i,50] = qc4[i,0]

        # Asselin Filter to control computational mode with Leapfrog in time
        if gamma>0:
            qf4[i-1,:] = qc4[i-1,:] + gamma*(qc4[i,:]-2*qc4[i-1,:]+qf4[i-2,:])
        else:
            qf4[i] = qc4[i]
    return qf4[:251]

In [7]:
def leapfrog_6th_order(f,cnum,gamma):
    qc6 = np.empty((251+1,51))
    qf6 = np.empty((251+1,51))
    qc6[0,:] = f.copy()
    qf6[0,:] = f.copy()

    # First time step, upwind
    qc6[1,0] = qc6[0,0] - cnum*(qc6[0,1]-qc6[0,49])
    qc6[1,1:50] = qc6[0,1:50] - cnum*(qc6[0,2:51]-qc6[0,0:49])
    qc6[1,50] = qc6[1,0]


    for i in range(2,251+1):
        qc6[i,0] = qf6[i-2,0] - cnum*2.*(45.*(qc6[i-1,1]-qc6[i-1,49]) + 9.*(qc6[i-1,48] - qc6[i-1,2]) 
                                       + qc6[i-1,3] - qc6[i-1,47])/60.
        qc6[i,1] = qf6[i-2,1] - cnum*2.*(45.*(qc6[i-1,2]-qc6[i-1,0]) + 9.*(qc6[i-1,49] - qc6[i-1,3]) 
                                       + qc6[i-1,4] - qc6[i-1,48])/60.
        qc6[i,2] = qf6[i-2,2] - cnum*2.*(45.*(qc6[i-1,3]-qc6[i-1,1]) + 9.*(qc6[i-1,0] - qc6[i-1,4]) 
                                       + qc6[i-1,5] - qc6[i-1,49])/60.

        qc6[i,3:48] = qf6[i-2,3:48] - cnum*2.*(45.*(qc6[i-1,4:49]-qc6[i-1,2:47]) + 9.*(qc6[i-1,1:46] - qc6[i-1,5:50]) 
                                           + qc6[i-1,6:51] - qc6[i-1,0:45])/60.
        # BCs
        qc6[i,48] = qf6[i-2,48] - cnum*2.*(45.*(qc6[i-1,49]-qc6[i-1,47]) + 9.*(qc6[i-1,46] - qc6[i-1,0])
                                         + qc6[i-1,1] - qc6[i-1,45])/60.
        qc6[i,49] = qf6[i-2,49] - cnum*2.*(45.*(qc6[i-1,0]-qc6[i-1,48]) + 9.*(qc6[i-1,47] - qc6[i-1,1])
                                         + qc6[i-1,1] - qc6[i-1,46])/60.
        qc6[i,50] = qc6[i,0]

        # Asselin Filter to control computational mode with Leapfrog in time
        if gamma>0:
            qf6[i-1,:] = qc6[i-1,:] + gamma*(qc6[i,:]-2*qc6[i-1,:]+qf6[i-2,:])
        else:
            qf6[i] = gc6[i]
    return qf6[:251]

In [8]:
def runge_kutta3_4th_order(f,cnum):
    rk34 = np.empty((251,51))
    rk3_1 = np.empty((251,51))
    rk3_2 = np.empty((251,51))
    rk34[0,:] = f
    rk3_1[0,:] = 0
    rk3_2[0,:] = 0

    for i in range(1,251):
        # First RK3 Step
        rk3_1[i,0] = rk34[i-1,0] - cnum/3.*(8.*(rk34[i-1,1]-rk34[i-1,49]) - rk34[i-1,2] + rk34[i-1,48])/12.
        rk3_1[i,1] = rk34[i-1,1] - cnum/3.*(8.*(rk34[i-1,2]-rk34[i-1,0]) - rk34[i-1,3] + rk34[i-1,49])/12.
        rk3_1[i,2:49] = rk34[i-1,2:49] - cnum/3.*(8.*(rk34[i-1,3:50]-rk34[i-1,1:48]) - rk34[i-1,4:51] 
                                                  + rk34[i-1,0:47])/12.

        # BCs
        rk3_1[i,49] = rk34[i-1,49] - cnum/3.*(8.*(rk34[i-1,0]-rk34[i-1,48]) - rk34[i-1,1] + rk34[i-1,47])/12.
        rk3_1[i,50] = rk3_1[i,0]

        # Second RK3 Step
        rk3_2[i,0] = rk34[i-1,0] - cnum/2.*(8.*(rk3_1[i,1]-rk3_1[i,49]) - rk3_1[i,2] + rk3_1[i,48])/12.
        rk3_2[i,1] = rk34[i-1,1] - cnum/2.*(8.*(rk3_1[i,2]-rk3_1[i,0]) - rk3_1[i,3] + rk3_1[i,49])/12.
        rk3_2[i,2:49] = rk34[i-1,2:49] - cnum/2.*(8.*(rk3_1[i,3:50]-rk3_1[i,1:48]) - rk3_1[i,4:51] 
                                                  + rk3_1[i,0:47])/12.

        # BCs
        rk3_2[i,49] = rk34[i-1,49] - cnum/2.*(8.*(rk3_1[i,0]-rk3_1[i,48]) - rk3_1[i,1] + rk3_1[i,47])/12.
        rk3_2[i,50] = rk3_2[i,0]

        # Third RK3 Step
        rk34[i,0] = rk34[i-1,0] - cnum*(8.*(rk3_2[i,1]-rk3_2[i,49]) - rk3_2[i,2] + rk3_2[i,48])/12.
        rk34[i,1] = rk34[i-1,1] - cnum*(8.*(rk3_2[i,2]-rk3_2[i,0]) - rk3_2[i,3] + rk3_2[i,49])/12.
        rk34[i,2:49] = rk34[i-1,2:49] - cnum*(8.*(rk3_2[i,3:50]-rk3_2[i,1:48]) - rk3_2[i,4:51] 
                                              + rk3_2[i,0:47])/12.

        # BCs
        rk34[i,49] = rk34[i-1,49] - cnum*(8.*(rk3_2[i,0]-rk3_2[i,48]) - rk3_2[i,1] + rk3_2[i,47])/12.
        rk34[i,50] = rk34[i,0]
    return rk34

In [9]:
def runge_kutta3_5th_order(f,cnum):
    rk35 = np.empty((251,51))
    rk3_1 = np.empty((251,51))
    rk3_2 = np.empty((251,51))
    rk35[0,:] = f.copy()
    rk3_1[0,:] = 0
    rk3_2[0,:] = 0


    for i in range(1,251):
        # First RK3 Step
        rk3_1[i,0] = rk35[i-1,0] - cnum/3.*(30.*rk35[i-1,1] - 60.*rk35[i-1,49] + 20.*rk35[i-1,0] + 15.*rk35[i-1,48] 
                                         - 3.*rk35[i-1,2] - 2.*rk35[i-1,47])/60.
        rk3_1[i,1] = rk35[i-1,1] - cnum/3.*(30.*rk35[i-1,2] - 60.*rk35[i-1,0] + 20.*rk35[i-1,1] + 15.*rk35[i-1,49] 
                                         - 3.*rk35[i-1,3] - 2.*rk35[i-1,48])/60.
        rk3_1[i,2] = rk35[i-1,2] - cnum/3.*(30.*rk35[i-1,3] - 60.*rk35[i-1,1] + 20.*rk35[i-1,2] + 15.*rk35[i-1,0]
                                         - 3.*rk35[i-1,4] - 2.*rk35[i-1,49])/60.

        rk3_1[i,3:48] = rk35[i-1,3:48] - cnum/3.*(30.*rk35[i-1,4:49] - 60.*rk35[i-1,2:47] + 20.*rk35[i-1,3:48] 
                                             + 15.*rk35[i-1,1:46] - 3.*rk35[i-1,5:50] - 2.*rk35[i-1,0:45])/60.
        # BCs
        rk3_1[i,48] = rk35[i-1,48] - cnum/3.*(30.*rk35[i-1,49] - 60.*rk35[i-1,47] + 20.*rk35[i-1,48] + 15.*rk35[i-1,46]
                                           - 3.*rk35[i-1,0] - 2.*rk35[i-1,45])/60.
        rk3_1[i,49] = rk35[i-1,49] - cnum/3.*(30.*rk35[i-1,0] - 60.*rk35[i-1,48] + 20.*rk35[i-1,49] + 15.*rk35[i-1,47]
                                           - 3.*rk35[i-1,1] - 2.*rk35[i-1,46])/60.
        rk3_1[i,50] = rk3_1[i,0]


        # Second RK3 Step
        rk3_2[i,0] = rk35[i-1,0] - cnum/2.*(30.*rk3_1[i,1] - 60.*rk3_1[i,49] + 20.*rk3_1[i,0] + 15.*rk3_1[i,48]
                                         - 3.*rk3_1[i,2] - 2.*rk3_1[i,47])/60.
        rk3_2[i,1] = rk35[i-1,1] - cnum/2.*(30.*rk3_1[i,2] - 60.*rk3_1[i,0] + 20.*rk3_1[i,1] + 15.*rk3_1[i,49] 
                                         - 3.*rk3_1[i,3] - 2.*rk3_1[i,48])/60.
        rk3_2[i,2] = rk35[i-1,2] - cnum/2.*(30.*rk3_1[i,3] - 60.*rk3_1[i,1] + 20.*rk3_1[i,2] + 15.*rk3_1[i,0]
                                         - 3.*rk3_1[i,4] - 2.*rk3_1[i,49])/60.

        rk3_2[i,3:48] = rk35[i-1,3:48] - cnum/2.*(30.*rk3_1[i,4:49] - 60.*rk3_1[i,2:47] + 20.*rk3_1[i,3:48] 
                                             + 15.*rk3_1[i,1:46] - 3.*rk3_1[i,5:50] - 2.*rk3_1[i,0:45])/60.
        # BCs
        rk3_2[i,48] = rk35[i-1,48] - cnum/2.*(30.*rk3_1[i,49] - 60.*rk3_1[i,47] + 20.*rk3_1[i,48] + 15.*rk3_1[i,46] 
                                          - 3.*rk3_1[i,0] - 2.*rk3_1[i,45])/60.
        rk3_2[i,49] = rk35[i-1,49] - cnum/2.*(30.*rk3_1[i,0] - 60.*rk3_1[i,48] + 20.*rk3_1[i,49] + 15.*rk3_1[i,47] 
                                          - 3.*rk3_1[i,1] - 2.*rk3_1[i,46])/60.
        rk3_2[i,50] = rk3_2[i,0]


        # Third RK3 Step
        rk35[i,0] = rk35[i-1,0] - cnum*(30.*rk3_2[i,1] - 60.*rk3_2[i,49] + 20.*rk3_2[i,0] + 15.*rk3_2[i,48] 
                                      - 3.*rk3_2[i,2] - 2.*rk3_2[i,47])/60.
        rk35[i,1] = rk35[i-1,1] - cnum*(30.*rk3_2[i,2] - 60.*rk3_2[i,0] + 20.*rk3_2[i,1] + 15.*rk3_2[i,49] 
                                      - 3.*rk3_2[i,3] - 2.*rk3_2[i,48])/60.
        rk35[i,2] = rk35[i-1,2] - cnum*(30.*rk3_2[i,3] - 60.*rk3_2[i,1] + 20.*rk3_2[i,2] + 15.*rk3_2[i,0] 
                                      - 3.*rk3_2[i,4] - 2.*rk3_2[i,49])/60.

        rk35[i,3:48] = rk35[i-1,3:48] - cnum*(30.*rk3_2[i,4:49] - 60.*rk3_2[i,2:47] + 20.*rk3_2[i,3:48] 
                                        + 15.*rk3_2[i,1:46] - 3.*rk3_2[i,5:50] - 2.*rk3_2[i,0:45])/60.
        # BCs
        rk35[i,48] = rk35[i-1,48] - cnum*(30.*rk3_2[i,49] - 60.*rk3_2[i,47] + 20.*rk3_2[i,48] + 15.*rk3_2[i,46]
                                        - 3.*rk3_2[i,0] - 2.*rk3_2[i,45])/60.
        rk35[i,49] = rk35[i-1,49] - cnum*(30.*rk3_2[i,0] - 60.*rk3_2[i,48] + 20.*rk3_2[i,49] + 15.*rk3_2[i,47]
                                        - 3.*rk3_2[i,1] - 2.*rk3_2[i,46])/60.
        rk35[i,50] = rk35[i,0]
    return rk35

In [10]:
def runge_kutta3_6th_order(f,cnum):
    rk36 = np.empty((251,51))
    rk3_1 = np.empty((251,51))
    rk3_2 = np.empty((251,51))
    rk36[0,:] = f.copy()
    rk3_1[0,:] = 0
    rk3_2[0,:] = 0


    for i in range(1,251):
        # First RK3 Step
        rk3_1[i,0] = rk36[i-1,0] - cnum/3.*(45.*(rk36[i-1,1]-rk36[i-1,49]) + 9.*(rk36[i-1,48] - rk36[i-1,2]) 
                                       + rk36[i-1,3] - rk36[i-1,47])/60.
        rk3_1[i,1] = rk36[i-1,1] - cnum/3.*(45.*(rk36[i-1,2]-rk36[i-1,0]) + 9.*(rk36[i-1,49] - rk36[i-1,3]) 
                                       + rk36[i-1,4] - rk36[i-1,48])/60.
        rk3_1[i,2] = rk36[i-1,2] - cnum/3.*(45.*(rk36[i-1,3]-rk36[i-1,1]) + 9.*(rk36[i-1,0] - rk36[i-1,4]) 
                                       + rk36[i-1,5] - rk36[i-1,49])/60.

        rk3_1[i,3:48] = rk36[i-1,3:48] - cnum/3.*(45.*(rk36[i-1,4:49]-rk36[i-1,2:47]) + 
                                                  9.*(rk36[i-1,1:46] - rk36[i-1,5:50]) + 
                                                  rk36[i-1,6:51] - rk36[i-1,0:45])/60.
        # BCs
        rk3_1[i,48] = rk36[i-1,48] - cnum/3.*(45.*(rk36[i-1,49]-rk36[i-1,47]) + 9.*(rk36[i-1,46] - rk36[i-1,0])
                                         + rk36[i-1,1] - rk36[i-1,45])/60.
        rk3_1[i,49] = rk36[i-1,49] - cnum/3.*(45.*(rk36[i-1,0]-rk36[i-1,48]) + 9.*(rk36[i-1,47] - rk36[i-1,1])
                                         + rk36[i-1,2] - rk36[i-1,46])/60.
        rk3_1[i,50] = rk3_1[i,0]


        # Second RK3 Step
        rk3_2[i,0] = rk36[i-1,0] - cnum/2.*(45.*(rk3_1[i,1]-rk3_1[i,49]) + 9.*(rk3_1[i,48] - rk3_1[i,2]) 
                                       + rk3_1[i,3] - rk3_1[i,47])/60.
        rk3_2[i,1] = rk36[i-1,1] - cnum/2.*(45.*(rk3_1[i,2]-rk3_1[i,0]) + 9.*(rk3_1[i,49] - rk3_1[i,3]) 
                                       + rk3_1[i,4] - rk3_1[i,48])/60.
        rk3_2[i,2] = rk36[i-1,2] - cnum/2.*(45.*(rk3_1[i,3]-rk3_1[i,1]) + 9.*(rk3_1[i,0] - rk3_1[i,4]) 
                                       + rk3_1[i,5] - rk3_1[i,49])/60.

        rk3_2[i,3:48] = rk36[i-1,3:48] - cnum/2.*(45.*(rk3_1[i,4:49]-rk3_1[i,2:47]) + 
                                                  9.*(rk3_1[i,1:46] - rk3_1[i,5:50]) + 
                                                  rk3_1[i,6:51] - rk3_1[i,0:45])/60.
        # BCs
        rk3_2[i,48] = rk36[i-1,48] - cnum/2.*(45.*(rk3_1[i,49]-rk3_1[i,47]) + 9.*(rk3_1[i,46] - rk3_1[i,0])
                                         + rk3_1[i,1] - rk3_1[i,45])/60.
        rk3_2[i,49] = rk36[i-1,49] - cnum/2.*(45.*(rk3_1[i,0]-rk3_1[i,48]) + 9.*(rk3_1[i,47] - rk3_1[i,1])
                                         + rk3_1[i,2] - rk3_1[i,46])/60.
        rk3_2[i,50] = rk3_2[i,0]


        # Third RK3 Step
        rk36[i,0] = rk36[i-1,0] - cnum*(45.*(rk3_2[i,1]-rk3_2[i,49]) + 9.*(rk3_2[i,48] - rk3_2[i,2]) 
                                       + rk3_2[i,3] - rk3_2[i,47])/60.
        rk36[i,1] = rk36[i-1,1] - cnum*(45.*(rk3_2[i,2]-rk3_2[i,0]) + 9.*(rk3_2[i,49] - rk3_2[i,3]) 
                                       + rk3_2[i,4] - rk3_2[i,48])/60.
        rk36[i,2] = rk36[i-1,2] - cnum*(45.*(rk3_2[i,3]-rk3_2[i,1]) + 9.*(rk3_2[i,0] - rk3_2[i,4]) 
                                       + rk3_2[i,5] - rk3_2[i,49])/60.

        rk36[i,3:48] = rk36[i-1,3:48] - cnum*(45.*(rk3_2[i,4:49]-rk3_2[i,2:47]) + 
                                              9.*(rk3_2[i,1:46] - rk3_2[i,5:50]) + 
                                              rk3_2[i,6:51] - rk3_2[i,0:45])/60.
        # BCs
        rk36[i,48] = rk36[i-1,48] - cnum*(45.*(rk3_2[i,49]-rk3_2[i,47]) + 9.*(rk3_2[i,46] - rk3_2[i,0])
                                         + rk3_2[i,1] - rk3_2[i,45])/60.
        rk36[i,49] = rk36[i-1,49] - cnum*(45.*(rk3_2[i,0]-rk3_2[i,48]) + 9.*(rk3_2[i,47] - rk3_2[i,1])
                                         + rk3_2[i,2] - rk3_2[i,46])/60.
        rk36[i,50] = rk36[i,0]
    return rk36

In [11]:
def run_advection(wave_type='Box',finite_diff='upwind',dt=400,gamma=0.03):

    c = 10      # Advecting wind 10 m/s 
    dx = 10000  # Grid spacing 10 km = 10000m
    
    cnum = c*dt/dx
    print('CFL Number =',cnum)
    
    x = np.linspace(0,1,51)
    
    if wave_type == 'Box':
        f = create_box(x)
    elif wave_type == 'Hill':
        f = create_hill(x)
    elif wave_type == 'Triangle':
        f = create_triangle(x)
    elif wave_type == 'Sawtooth Box':
        f = create_sawtooth_box(x)
    elif wave_type == 'Sawtooth Triangle':
        f = create_sawtooth_triangle(x)
    elif wave_type == 'Sine Wave':
        f = create_sine_wave(x)
    
    exact_f = np.empty((251,51))
    f2 = f.copy()
    exact_f[0,:] = f2
    roll = 0
    for n in range(1,251):
        a = cnum/50*n
        if a>1:
            a = a - int(a)
        if np.rint(a*100) in (x*100).astype('int'):
            roll = np.where(np.rint(a*100) == (x*100).astype('int'))[0][0]
            if roll > 32: roll+=1
            f2 = np.roll(f,roll)
        exact_f[n,:] = f2
    
    plt.close()

    if finite_diff == 'upwind':
        pred = forward_upwind(f,cnum)
        title = 'Forward in Time, Upstream in Space'
    elif finite_diff == 'leapfrog2':
        pred = leapfrog_2nd_order(f,cnum,gamma)
        title = 'Leapfrog in time, Leapfrog 2nd Order in Space'
    elif finite_diff == 'leapfrog4':
        pred = leapfrog_4th_order(f,cnum,gamma)
        title = 'Leapfrog in time, Leapfrog 4th Order in Space'
    elif finite_diff == 'leapfrog6':
        pred = leapfrog_6th_order(f,cnum,gamma)
        title = 'Leapfrog in time, Leapfrog 6th Order in Space'
    elif finite_diff == 'rk34':
        pred = runge_kutta3_4th_order(f,cnum)
        title = 'Runge-Kutta3 in Time and 4th Order Space'
    elif finite_diff == 'rk35':
        pred = runge_kutta3_5th_order(f,cnum)
        title = 'Runge-Kutta3 in Time and 5th Order Space'
    elif finite_diff == 'rk36':
        pred = runge_kutta3_6th_order(f,cnum)
        title = 'Runge-Kutta3 in Time and 6th Order Space'

    anim = animate(pred, exact_f, x, f, title)

    display(anim)

In [13]:
from ipywidgets import interact_manual, widgets

print('Welcome to 1D Advection Du Jour')
print()
print('dx = 10 km = 10000 m')
print(' c = 10 m/s')
interact_manual(run_advection,
                wave_type=['Box','Triangle','Sawtooth Box','Sawtooth Triangle','Hill','Sine Wave'],
                finite_diff={'Forward Time; Upwind Space':'upwind',
                             'Leapfrog Time; 2nd Order Space':'leapfrog2',
                             'Leapfrog Time; 4th Order Space':'leapfrog4',
                             'Leapfrog Time; 6th Order Space':'leapfrog6',
                             'RK3 Time; 4th Order Space':'rk34',
                             'RK3 Time; 5th Order Space':'rk35',
                             'RK3 Time; 6th Order Space':'rk36'},
                dt=range(100,1401,100), 
                gamma=(0.0,0.5,.01));

Welcome to 1D Advection Du Jour

dx = 10 km = 10000 m
 c = 10 m/s


interactive(children=(Dropdown(description='wave_type', options=('Box', 'Triangle', 'Sawtooth Box', 'Sawtooth Triangle', 'Hill', 'Sine Wave'), value='Box'), Dropdown(description='finite_diff', options={'Forward Time; Upwind Space': 'upwind', 'Leapfrog Time; 2nd Order Space': 'leapfrog2', 'Leapfrog Time; 4th Order Space': 'leapfrog4', 'Leapfrog Time; 6th Order Space': 'leapfrog6', 'RK3 Time; 4th Order Space': 'rk34', 'RK3 Time; 5th Order Space': 'rk35', 'RK3 Time; 6th Order Space': 'rk36'}, value='upwind'), Dropdown(description='dt', index=3, options=(100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400), value=400), FloatSlider(value=0.03, description='gamma', max=0.5, step=0.01), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))